<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/PyTorch/Advanced/ProtXLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtXLNet pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 1.1MB 2.6MB/s 
     |████████████████████████████████| 3.0MB 8.2MB/s 
     |████████████████████████████████| 890kB 34.0MB/s 
     |████████████████████████████████| 1.1MB 33.2MB/s 


In [2]:
import torch
from transformers import XLNetModel, XLNetTokenizer
import re
import os
import requests
from tqdm.auto import tqdm

<b>2. Load the vocabulary and ProtXLNet Model<b>

In [3]:
tokenizer = XLNetTokenizer.from_pretrained("Rostlab/prot_xlnet", do_lower_case=False)

In [4]:
xlnet_men_len = 512

In [5]:
model = XLNetModel.from_pretrained("Rostlab/prot_xlnet",mem_len=xlnet_men_len)

/usr/local/lib/python3.6/dist-packages/transformers/configuration_xlnet.py:212: FutureWarning: This config doesn't use attention memories, a core feature of XLNet. Consider setting `men_len` to a non-zero value, for example `xlnet = XLNetLMHeadModel.from_pretrained('xlnet-base-cased'', mem_len=1024)`, for accurate training performance as well as an order of magnitude faster inference. Starting from version 3.5.0, the default parameter will be 1024, following the implementation in https://arxiv.org/abs/1906.08237
  FutureWarning,


<b>3. Load the model into the GPU if avilabile and switch to inference mode<b>

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
model = model.to(device)
model = model.eval()

<b>4. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (unk)<b>

In [8]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [9]:
sequences_Example = [re.sub(r"[UZOB]", "<unk>", sequence) for sequence in sequences_Example]

<b>5. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [10]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [11]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

<b>6. Extracting sequences' features and load it into the CPU if needed<b>

In [12]:
with torch.no_grad():
    embedding, memory = model(input_ids=input_ids,attention_mask=attention_mask,mems=None)

In [13]:
embedding = embedding.cpu().numpy()

<b>7. Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by ProtXLNet model<b>

In [14]:
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    padded_seq_len = len(attention_mask[seq_num])
    seq_emd = embedding[seq_num][padded_seq_len-seq_len:padded_seq_len-2]
    features.append(seq_emd)

In [15]:
print(features)

[array([[ 0.48745167, -0.7708792 ,  0.99001795, ..., -0.37356147,
        -1.0589762 ,  0.95599836],
       [ 0.21325932, -0.5478907 ,  0.61154914, ..., -0.05471409,
        -0.87878996,  0.24645182],
       [ 0.37891346, -0.63965833,  0.67224485, ..., -0.14891213,
        -0.6769578 ,  0.34598225],
       ...,
       [ 0.09265022, -0.6810118 ,  0.5181867 , ..., -0.32756206,
        -0.56731236, -0.1644128 ],
       [-0.08533159, -0.7438235 ,  0.29890788, ..., -0.24376422,
        -0.11153184, -0.72609985],
       [-0.4822567 , -0.83816946,  0.08214366, ..., -0.25196356,
        -0.03577508, -0.5348949 ]], dtype=float32), array([[ 1.0403055 , -0.9504928 ,  0.33534604, ..., -0.23747401,
        -0.27550647,  0.47948697],
       [ 0.5085353 , -0.9508467 ,  1.0235132 , ..., -0.05893134,
        -0.9752796 ,  0.08713176],
       [ 0.6562663 , -0.75406194,  0.43234664, ...,  0.36707622,
        -0.7063239 , -0.555342  ],
       [ 0.59033847,  0.14151998,  0.2957885 , ...,  0.20600991,
     